IBS= (Close-Low)/(High-Low)
 
We simply buy on close if IBS is lower than 0.2, and we sell any day later when the IBS closes above 0.8.

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import date, timedelta

c:\Users\cheng\anaconda3\Lib\site-packages\yfinance\base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


In [2]:
end_date = date.today() - timedelta(days=1)
start_date = end_date - timedelta(days=50)

In [8]:
start_date = "2023-01-01"
end_date = "2024-01-01"

In [9]:
sp500_hist = yf.Ticker('^GSPC').history(start=start_date, end=end_date)
spy_hist = yf.Ticker('SPY').history(start=start_date, end=end_date)
voo_hist = yf.Ticker('VOO').history(start=start_date, end=end_date)

In [15]:
sp500_hist.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2023-01-03 00:00:00-05:00,3853.290039,3878.459961,3794.330078,3824.139893,3959140000,0.0,0.0
2023-01-04 00:00:00-05:00,3840.360107,3873.159912,3815.770020,3852.969971,4414080000,0.0,0.0
2023-01-05 00:00:00-05:00,3839.739990,3839.739990,3802.419922,3808.100098,3893450000,0.0,0.0
2023-01-06 00:00:00-05:00,3823.370117,3906.189941,3809.560059,3895.080078,3923560000,0.0,0.0
2023-01-09 00:00:00-05:00,3910.820068,3950.570068,3890.419922,3892.090088,4311770000,0.0,0.0


In [11]:
ibs_value = (sp500_hist['Close'] - sp500_hist['Low']) / (sp500_hist['High'] - sp500_hist['Low'])
ibs = pd.DataFrame(ibs_value.reset_index())
ibs['Date'] = ibs['Date'].dt.strftime('%Y-%m-%d')
ibs.columns = ["Date", "IBS"]
ibs['Decision'] = np.select([(ibs['IBS'] > 0.8), (ibs['IBS'] < 0.2)], ['Sell', 'Buy'], default='')

In [12]:
buy_condition = ibs['Decision'] == 'Buy'
sell_condition = ibs['Decision'] == 'Sell'

ibs['SPY Result'] = np.select([buy_condition, sell_condition], [
    spy_hist['Open'].shift(-1) - spy_hist['Close'],
    spy_hist['Close'] - spy_hist['Open'].shift(-1)
], default=np.nan)

ibs['VOO Result'] = np.select([buy_condition, sell_condition], [
    voo_hist['Open'].shift(-1) - voo_hist['Close'],
    voo_hist['Close'] - voo_hist['Open'].shift(-1)
], default=np.nan)

ibs.head(10)

,Date,IBS,Decision,SPY Result,VOO Result
0,2023-01-03,0.354331,,NaN,NaN
1,2023-01-04,0.648197,,NaN,NaN
2,2023-01-05,0.152202,Buy,3.181654,2.883600
3,2023-01-06,0.885027,Sell,-2.255696,-2.106139
4,2023-01-09,0.027767,Buy,-0.600872,-0.482243
5,2023-01-10,0.986364,Sell,-1.625365,-1.515626
6,2023-01-11,0.988925,Sell,-1.132820,-1.013708
7,2023-01-12,0.757640,,NaN,NaN
8,2023-01-13,0.913649,Sell,0.019694,0.039360
9,2023-01-17,0.207655,,NaN,NaN


In [13]:
print("SPY: ", ibs["SPY Result"].mean())
print("VOO: ", ibs["VOO Result"].mean())

SPY:  0.06935849425627837
VOO:  0.05302864616772803
